In [ ]:
email_conversation = """Dear Axone Support Team,

We are currently using the VideoViewerNextGen module (v1.8.35) in Ignition Perspective.
In our environment, some NVR/CCTV cameras provide streams in H.264, while others randomly switch to H.265 depending on the device or channel settings.

At the moment, our pipeline is configured as follows:

rtspsrc location="%s" protocols=tcp latency=300 do-rtsp-keep-alive=true retry=10 \timeout=10000000 tcp-timeout=5000000 ! \rtph264depay ! h264parse ! avdec_h264 ! videoscale ! videoconvert ! \x264enc speed-preset=ultrafast tune=zerolatency bitrate=2000 ! \rtph264pay config-interval=1 pt=96 ! application/x-rtp,media=video,encoding-name=H264,payload=96 ! \webrtcbin name=webrtcbin bundle-policy=max-bundle
This configuration works only when the incoming stream is H.264.
However, when the stream is H.265, the pipeline fails (no video is displayed, errors are raised).

We would like to ask:

Does the VideoViewerNextGen module support automatic codec detection (H.264 vs H.265) within the pipeline?

If not, what is the recommended approach when the codec may vary per stream?

Should we build a dynamic pipeline (separate branches for H.264 and H.265)?

Or is there any built-in option in the module to handle mixed codec environments?

Is there a configuration parameter to force transcoding of H.265 → H.264 for WebRTC, since most browsers do not support H.265?

We would greatly appreciate your guidance on best practices in this scenario.

Best regards,
shwang
"""

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
from pydantic import BaseModel, Field


class EmailSummary(BaseModel):
    person: str = Field(description="이메일을 보낸 사람")
    company: str = Field(description="이메일을 보낸 사람의 회사")
    date: str = Field(description="이메일을 보낸 날짜")
    time: str = Field(description="이메일을 보낸 시간")
    purpose: str = Field(description="이메일의 목적")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")

In [20]:
## LCEL 구조

# chain = prompt | llm | output_parser

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [ ]:
from langchain.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

#QUESTION:
다음의 이메일 내용 중에서 주요 내용을 추출해 주세요.

#EMAIL CONVERSATION:
{email_conversation}

#FORMAT:
{format}
"""
)

prompt = prompt.partial(format=output_parser.get_format_instructions())

In [24]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "이메일을 보낸 사람", "title": "Person", "type": "string"}, "company": {"description": "이메일을 보낸 사람의 회사", "title": "Company", "type": "string"}, "date": {"description": "이메일을 보낸 날짜", "title": "Date", "type": "string"}, "time": {"description": "이메일을 보낸 시간", "title": "Time", "type": "string"}, "purpose": {"description": "이메일의 목적", "title": "Purpose", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type"

In [14]:
chain = prompt | llm | output_parser

In [ ]:
answer = chain.invoke({"email_conversation": email_conversation})

In [29]:
print(answer.summary)

현재 Ignition Perspective에서 VideoViewerNextGen 모듈(v1.8.35)을 사용 중이며, 일부 NVR/CCTV 카메라가 H.264와 H.265 스트림을 제공하고 있습니다. 현재 파이프라인은 H.264 스트림에서만 작동하며, H.265 스트림에서는 실패합니다. VideoViewerNextGen 모듈이 파이프라인 내에서 코덱 자동 감지를 지원하는지, 그렇지 않다면 스트림별로 코덱이 다를 때의 권장 접근 방식은 무엇인지, H.264와 H.265를 위한 동적 파이프라인을 구축해야 하는지, 혼합 코덱 환경을 처리할 수 있는 모듈 내 옵션이 있는지, WebRTC를 위한 H.265 → H.264 트랜스코딩을 강제하는 구성 매개변수가 있는지에 대해 문의드립니다.


In [30]:
print(answer.purpose)

VideoViewerNextGen 모듈의 코덱 자동 감지 및 혼합 코덱 환경 처리 방법 문의


In [31]:
print(answer.person)

shwang
